# [Concurrency](https://en.wikipedia.org/wiki/Concurrency_(computer_science)
It is the possibility of running code in parallel with other codes.

## 1. Threads
Parallel execution of code which can share objects easely.

In [3]:
import threading
import time
import sys

a = ''

def task_1():
    global a
    for i in range(10):
        print('o', end='')
        sys.stdout.flush()
        a += 'o'
        time.sleep(1)
        
def task_2():
    global a
    for i in range(20):
        print('O', end='')
        sys.stdout.flush()
        a += 'O'
        time.sleep(0.6)
        
thread_1 = threading.Thread(target=task_1)
thread_2 = threading.Thread(target=task_2)

thread_1.start()
thread_2.start()
print("(Both threads have started)")

thread_1.join()
thread_2.join()
print("\nBoth threads have finished")

print(a)

o(Both threads have started)O
OoOOoOoOOoOOoOoOOoOOoOoOOOOO
Both threads have finished
oOOoOOoOoOOoOOoOoOOoOOoOoOOOOO


[CPU-bound tasks are not a good fit for Python threads, due to the Global Interpreter Lock (GIL). Parallel computations in Python should be done in multiple processes, not threads.](http://eli.thegreenplace.net/2011/12/27/python-threads-communication-and-stopping) As a consequence of this problem, currently it is impossible to use more than one core using Python Threads, exclusively.

## Processes
Parallel execution of code that have a larger CPU overhead when sharing objects. Supported by the OS.

In [4]:
import multiprocessing
import time
import sys

a = ''

def task_1():
    global a
    for i in range(10):
        print('o', end='')
        sys.stdout.flush()
        a += 'o'
        time.sleep(1)
        
def task_2():
    global a
    for i in range(20):
        print('O', end='')
        sys.stdout.flush()
        a += 'O'
        time.sleep(0.6)
        
process_1 = multiprocessing.Process(target=task_1)
process_2 = multiprocessing.Process(target=task_2)

process_1.start()
process_2.start()
print("(Both processes have started)")

process_1.join()
process_2.join()
print("\nBoth processes have finished")

print(a)

oO(Both processes have started)
OoOOoOoOOoOOoOoOOoOOoOoOOOOO
Both processes have finished



### Why `a` has not been modified?

By definition, processes must [__fork__](https://en.wikipedia.org/wiki/Fork_(system_call) (make a copy of itself, that is, the code and the used memory) before start running.

## Using critical sections
A critical section is a region of code that should not run in parallel.

In [ ]:
# Two threads that have a critical section executed in parallel
import threading
import time
import sys

counter = 10

def task_1():
    global counter
    for i in range(1000000):
        counter += 1
        
def task_2():
    global counter
    for i in range(1000001):
        counter -= 1
        
thread_1 = threading.Thread(target=task_1)
thread_2 = threading.Thread(target=task_2)

thread_1.start()
thread_2.start()
print("(Both threads started)")

thread_1.join()
thread_2.join()
print("\nBoth threads finished")

print('counter =', counter)

In [ ]:
# Two threads that have a critical section executed sequentially
import threading
import time
import sys

lock = threading.Lock()
counter = 10

def task_1():
    global counter
    for i in range(1000000):
        with lock:
            counter += 1
        
def task_2():
    global counter
    for i in range(1000001):
        with lock:
            counter -= 1
        
thread_1 = threading.Thread(target=task_1)
thread_2 = threading.Thread(target=task_2)

thread_1.start()
thread_2.start()
print("(Both threads started)")

thread_1.join()
thread_2.join()
print("\nBoth threads finished")

print('counter =', counter)

## Synchronizing parallel tasks

Using a queue.

In [ ]:
import threading
import time
import random
import queue

queue_length = 10
q = queue.Queue(queue_length)

class Producer(threading.Thread):
    def __init__(self, iters=10):
        super(Producer,self).__init__()
        self.iters = iters

    def run(self):
        i = 0
        while i < self.iters:
            if not q.full():
                item = random.randint(1,10)
                q.put(item)
                print('Produced {} (queue length = {})'.format(item,q.qsize()))
                i += 1
                time.sleep(random.random())
        return

class Consumer(threading.Thread):
    def __init__(self, iters=10):
        super(Consumer,self).__init__()
        self.iters = iters

    def run(self):
        i = 0
        while i < self.iters:
            if not q.empty():
                item = q.get()
                print('Consumed {} (queue length = {})'.format(item,q.qsize()))
                i += 1
                time.sleep(random.random())
        return

Producer().start()
Consumer().start()

Using a socket.

## Coroutines (systolic computation)

In [32]:
def minimize():
    current = yield
    while True:
        value = yield current # Receives "value" and returns "current"
        current = min(value, current)
        
it = minimize()
next(it)            # Prime the coroutine
print(it.send(10))
print(it.send(4))
print(it.send(22))
print(it.send(-1))

10
4
4
-1


In [ ]:
def task_1():
    while True:
        received = yield
        print('1', received)
        
def task_2(other):
    while True:
        received = yield
        other.send(2)
        print('2', received)

t1 = task_1(); next(t1)
t2 = task_2(t1); next(t2)
print('next executed')
for i in range(10):
    t2.send(3)

In [19]:
def task_1():
    while True:
        received = yield
        print('1', received)
        
def task_2(other):
    while True:
        received = yield
        other.send(2)
        print('2', received)

t1 = task_1(); next(t1)
t2 = task_2(t1); next(t2)
print('next executed')
for i in range(3):
    t2.send(3)

next executed
1 2
2 3
1 2
2 3
1 2
2 3


In [1]:
# http://code.activestate.com/recipes/578265-pipeline-made-of-coroutines/
def coroutine(function):
    '''Performs the mandatory initial "next" on a coroutine.'''
    def generator(*args, **kwargs):
        primed_func = function(*args, **kwargs)
        primed_func.__next__()
        return primed_func
    return generator

In [39]:
@coroutine
def add(x):
    '''Add "x" to the input.'''
    output = yield
    while True:
        received = yield output
        output = received + x
        
@coroutine
def divide(other_task, x):
    '''Divide the input by "x".'''
    output = yield
    while True:
        received = yield output
        output = other_task.send(received / x)

pipeline = divide(add(1),2)
for i in range(10):
    print(i, pipeline.send(i))

0 0
1 0.5
2 2.0
3 2.5
4 3.0
5 3.5
6 4.0
7 4.5
8 5.0
9 5.5


In [ ]:
https://www.youtube.com/watch?v=Bv25Dwe84g0